In [1]:
import requests
import bs4
from bs4 import BeautifulSoup
from urllib.parse import urljoin
import pandas as pd

In [58]:
url_inicial = 'https://hidrobiologica.izt.uam.mx/index.php/revHidro/issue/archive'
url_root = 'https://hidrobiologica.izt.uam.mx/index.php/revHidro/issue/archive'
r=requests.get(url_inicial)

In [59]:
soup = BeautifulSoup(r.text, 'html.parser')

In [60]:
box = soup.find('ul', class_='issues_archive')
volumen=box.findAll('div', class_='obj_issue_summary')
vol = [x.find('a').get('href')for x in volumen]
vol

['https://hidrobiologica.izt.uam.mx/index.php/revHidro/issue/view/104',
 'https://hidrobiologica.izt.uam.mx/index.php/revHidro/issue/view/103',
 'https://hidrobiologica.izt.uam.mx/index.php/revHidro/issue/view/102',
 'https://hidrobiologica.izt.uam.mx/index.php/revHidro/issue/view/101',
 'https://hidrobiologica.izt.uam.mx/index.php/revHidro/issue/view/100',
 'https://hidrobiologica.izt.uam.mx/index.php/revHidro/issue/view/99',
 'https://hidrobiologica.izt.uam.mx/index.php/revHidro/issue/view/98',
 'https://hidrobiologica.izt.uam.mx/index.php/revHidro/issue/view/97',
 'https://hidrobiologica.izt.uam.mx/index.php/revHidro/issue/view/95',
 'https://hidrobiologica.izt.uam.mx/index.php/revHidro/issue/view/94',
 'https://hidrobiologica.izt.uam.mx/index.php/revHidro/issue/view/93',
 'https://hidrobiologica.izt.uam.mx/index.php/revHidro/issue/view/92',
 'https://hidrobiologica.izt.uam.mx/index.php/revHidro/issue/view/88',
 'https://hidrobiologica.izt.uam.mx/index.php/revHidro/issue/view/86',
 

In [61]:
vol2 =[]
for i in vol: 
    url_inicial1=i 
    r1 = requests.get(url_inicial1)
    soup1 = BeautifulSoup(r1.text, 'html.parser')
    box1 = soup1.find('div', class_='obj_issue_toc') 
    volumen1=box1.findAll('div', class_='obj_article_summary')
    vol1 = [x.find('a').get('href')for x in volumen1]
    vol2+=vol1

In [62]:
vol2

['https://hidrobiologica.izt.uam.mx/index.php/revHidro/article/view/1638',
 'https://hidrobiologica.izt.uam.mx/index.php/revHidro/article/view/1363',
 'https://hidrobiologica.izt.uam.mx/index.php/revHidro/article/view/1502',
 'https://hidrobiologica.izt.uam.mx/index.php/revHidro/article/view/1565',
 'https://hidrobiologica.izt.uam.mx/index.php/revHidro/article/view/1579',
 'https://hidrobiologica.izt.uam.mx/index.php/revHidro/article/view/1593',
 'https://hidrobiologica.izt.uam.mx/index.php/revHidro/article/view/1595',
 'https://hidrobiologica.izt.uam.mx/index.php/revHidro/article/view/1582',
 'https://hidrobiologica.izt.uam.mx/index.php/revHidro/article/view/1621',
 'https://hidrobiologica.izt.uam.mx/index.php/revHidro/article/view/1396',
 'https://hidrobiologica.izt.uam.mx/index.php/revHidro/article/view/1433',
 'https://hidrobiologica.izt.uam.mx/index.php/revHidro/article/view/1558',
 'https://hidrobiologica.izt.uam.mx/index.php/revHidro/article/view/1566',
 'https://hidrobiologica.

In [63]:
def get_url_items(sopa,url):
    box1 = soup1.find('div', class_='obj_issue_toc') 
    volumen1=box1.findAll('div', class_='obj_article_summary')
    vol1 = [x.find('a').get('href')for x in volumen1]
    return vol2

In [64]:
links_items=[]
i=0
while i<1:
    i+=1
    print(f'Estoy en la pagina {url_inicial}')
    r_pag=requests.get(url_inicial)
    s_p=BeautifulSoup(r_pag.text,'html.parser')
    links=get_url_items(s_p, url_inicial)
    links_items.append(links) 
    next_a=s_p.select('li.next > a')
    if not next_a or not next_a[0].get('href'):
        break

Estoy en la pagina https://hidrobiologica.izt.uam.mx/index.php/revHidro/issue/archive


In [65]:
list_scraper=[]
for i in links_items:
    for j in i:
        list_scraper.append(j)
len(list_scraper)

327

In [66]:
uno=list_scraper[0]
r_item=requests.get(uno)
s_item=BeautifulSoup(r_item.text, 'html.parser')

In [67]:
#funcion para iniciar el escraper de cada libro
def scraper_book(url):
    content_book={}
    r=requests.get(url)
    tire='Hidrobiológica'
    a='I. Físico Matemáticas y Ciencias de la Tierra'
    tem='Recursos hidrobiológicos, aspectos fisiológicos y biológicos de los ambientes acuáticos.'
    s_item=BeautifulSoup(r.text,'html.parser')
 #titulo de revista
    try:
        titulo=tire
        content_book['Titulo Revista']=tire
    except AttributeError:
        content_book['Titulo Revista']=None
    #area
    try:
        area=a
        content_book['Area']=a
    except AttributeError:
        content_book['Area']=None
    #tematica
    try:
        tema=tem
        content_book['Tematica']=tem
    except AttributeError:
        content_book['Tematica']=None
    #titulo articulo
    try:
        titu=s_item.find('h1', class_='page_title').get_text(strip=True)
        content_book['Titulo Articulo']=titu
    except AttributeError:
        content_book['Titulo Articulo']=None
    #resumen
    try:
        resu=s_item.find('div', class_='item abstract').get_text(strip=True)
        content_book['Resumen']=resu.replace("Resumen","")
    except AttributeError:
        content_book['Resumen']=None
    #abstract
    try:
        abst=s_item.find('div', class_='item cover_image').get_text(strip=True)
        content_book['Abstract']=abst
    except AttributeError:
        content_book['Abstract']=None
    #Link incial
    try:
        linkart=url
        content_book['Link Articulo']=linkart
    except AttributeError:
        content_book['Link Articulo']=None
    #link articulo
    try:
        link=s_item.find('a', class_='obj_galley_link pdf').get('href')
        content_book['Link PDF']=urljoin(url_root, link)
    except AttributeError:
        content_book['Link PDF']=None
    return content_book

In [68]:
list_scraper=list_scraper[0:327]
datos_book=[]
for idx, i in enumerate(list_scraper):
    print(f'estas escrapeando la pag {idx}')
    datos_book.append(scraper_book(i))

estas escrapeando la pag 0
estas escrapeando la pag 1
estas escrapeando la pag 2
estas escrapeando la pag 3
estas escrapeando la pag 4
estas escrapeando la pag 5
estas escrapeando la pag 6
estas escrapeando la pag 7
estas escrapeando la pag 8
estas escrapeando la pag 9
estas escrapeando la pag 10
estas escrapeando la pag 11
estas escrapeando la pag 12
estas escrapeando la pag 13
estas escrapeando la pag 14
estas escrapeando la pag 15
estas escrapeando la pag 16
estas escrapeando la pag 17
estas escrapeando la pag 18
estas escrapeando la pag 19
estas escrapeando la pag 20
estas escrapeando la pag 21
estas escrapeando la pag 22
estas escrapeando la pag 23
estas escrapeando la pag 24
estas escrapeando la pag 25
estas escrapeando la pag 26
estas escrapeando la pag 27
estas escrapeando la pag 28
estas escrapeando la pag 29
estas escrapeando la pag 30
estas escrapeando la pag 31
estas escrapeando la pag 32
estas escrapeando la pag 33
estas escrapeando la pag 34
estas escrapeando la pag 35
es

In [69]:
df_catalogo=pd.DataFrame(datos_book)
df_catalogo

,Titulo Revista,Area,Tematica,Titulo Articulo,Resumen,Abstract,Link Articulo,Link PDF
0,Hidrobiológica,I. Físico Matemáticas y Ciencias de la Tierra,"Recursos hidrobiológicos, aspectos fisiológico...",Preliminares,None,,https://hidrobiologica.izt.uam.mx/index.php/re...,https://hidrobiologica.izt.uam.mx/index.php/re...
1,Hidrobiológica,I. Físico Matemáticas y Ciencias de la Tierra,"Recursos hidrobiológicos, aspectos fisiológico...",Variabilidad iónica en la red hidrográfica de ...,"Antecedentes. Factores como la litología, la g...",,https://hidrobiologica.izt.uam.mx/index.php/re...,https://hidrobiologica.izt.uam.mx/index.php/re...
2,Hidrobiológica,I. Físico Matemáticas y Ciencias de la Tierra,"Recursos hidrobiológicos, aspectos fisiológico...",Variación estacional en la distribución y dive...,"Antecedentes. En el Golfo de México, el Sistem...",,https://hidrobiologica.izt.uam.mx/index.php/re...,https://hidrobiologica.izt.uam.mx/index.php/re...
3,Hidrobiológica,I. Físico Matemáticas y Ciencias de la Tierra,"Recursos hidrobiológicos, aspectos fisiológico...","Efecto embriotóxico, teratogénico y genotóxico...",Antecedentes. La comprensión del mecanismo de ...,,https://hidrobiologica.izt.uam.mx/index.php/re...,https://hidrobiologica.izt.uam.mx/index.php/re...
4,Hidrobiológica,I. Físico Matemáticas y Ciencias de la Tierra,"Recursos hidrobiológicos, aspectos fisiológico...",Evaluación y estimación de los puntos de refer...,Antecedentes. La jaiba azul Callinectes sapidu...,,https://hidrobiologica.izt.uam.mx/index.php/re...,https://hidrobiologica.izt.uam.mx/index.php/re...
...,...,...,...,...,...,...,...,...
322,Hidrobiológica,I. Físico Matemáticas y Ciencias de la Tierra,"Recursos hidrobiológicos, aspectos fisiológico...",Adiciones a la taxonomía de Brachionus josefin...,"En el estado de Aguascalientes, México, se enc...",,https://hidrobiologica.izt.uam.mx/index.php/re...,https://hidrobiologica.izt.uam.mx/index.php/re...
323,Hidrobiológica,I. Físico Matemáticas y Ciencias de la Tierra,"Recursos hidrobiológicos, aspectos fisiológico...",Primer registro y establecimiento del bagre de...,Se reporta la introducción de Ictalurus puncta...,,https://hidrobiologica.izt.uam.mx/index.php/re...,https://hidrobiologica.izt.uam.mx/index.php/re...
324,Hidrobiológica,I. Físico Matemáticas y Ciencias de la Tierra,"Recursos hidrobiológicos, aspectos fisiológico...",First record of the dusky grouper Epinephelus ...,The dusky grouper Epinephelus marginatus is a ...,,https://hidrobiologica.izt.uam.mx/index.php/re...,https://hidrobiologica.izt.uam.mx/index.php/re...
325,Hidrobiológica,I. Físico Matemáticas y Ciencias de la Tierra,"Recursos hidrobiológicos, aspectos fisiológico...",índice del volumen,None,,https://hidrobiologica.izt.uam.mx/index.php/re...,https://hidrobiologica.izt.uam.mx/index.php/re...


In [70]:
df_catalogo.to_csv('Revista041.01.csv', index=False)